In [1]:
from utils import *
from sklearn.ensemble import ExtraTreesClassifier
from itertools import product

/Users/abk0005/Competitions/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/abk0005/Competitions/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/abk0005/Competitions/env/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = aggregate(lb_encode = False)
df = clean_data(df)

--------------------
train & test ( Sun Sep 16 11:57:06 2018 )
--------------------
['department', 'region', 'education', 'gender', 'recruitment_channel', 'no_of_trainings', 'previous_year_rating', 'KPIs_met >80%', 'awards_won?', 'length_of_service', 'age', 'KPI_award', 'region_channel', 'education_region']
     DF shape: (78292, 443)
After removing empty features there are 439 features
After removing features with the same distribution on 0 and 1 classes there are 216 features
After removing features with not the same distribution on train and test datasets there are 216 features
After removing features which are having information value less than 0.02 and greater than 0.8s 211 features


In [21]:
def cv_scores(estimator, df, num_folds, params, stratified = False, verbose = 100, random_state = 11,
              save_train_prediction = True, train_prediction_file_name = 'train_prediction.csv',
              save_test_prediction = True, test_prediction_file_name = 'test_prediction.csv', iteration = 0,
              logger=None, lgbm_clf = True):
    warnings.simplefilter('ignore')
    
    if hasattr(estimator.get_params(), 'random_seed'):
        params['random_seed'] = np.random.choice(1000, 1)[0]
        
    if hasattr(estimator.get_params(), 'random_state'):
        params['random_state'] = np.random.choice(1000, 1)[0]

    clf = estimator.set_params(**params)
    
    feats = [f for f in df.columns if f not in ['index', 'is_promoted', 'employee_id']]
    
    for f in feats:
        df[f].fillna(df[f].mean(), inplace = True)
    
    # Divide in training/validation and test data
    
    train_df = df[df['is_promoted'].notnull()]
    test_df = df[df['is_promoted'].isnull()]
    print("Starting Classifier. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits = num_folds, shuffle = True, random_state = random_state)
    else:
        folds = KFold(n_splits = num_folds, shuffle = True, random_state = random_state)
        
    # Create arrays and dataframes to store results
    train_pred = np.zeros(train_df.shape[0])
    train_pred_proba = np.zeros(train_df.shape[0])

    test_pred = np.zeros(train_df.shape[0])
    test_pred_proba = np.zeros(train_df.shape[0])
    
    prediction_proba = np.zeros(test_df.shape[0], dtype= np.float16)
    
    df_feature_importance = pd.DataFrame(index = feats)
    folds_score = []
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['is_promoted'])):
        print('Fold', n_fold, 'started at', time.ctime())
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['is_promoted'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['is_promoted'].iloc[valid_idx]
        cat_feats_index = [i for i,c in enumerate(train_x.columns) if '_encoded' in c]
        
        clf.fit(train_x, train_y)
            
        if lgbm_clf:
            train_pred[train_idx] = clf.predict(train_x, num_iteration = clf.best_iteration_)
            train_pred_proba[train_idx] = clf.predict_proba(train_x, num_iteration = clf.best_iteration_)[:, 1]
            test_pred[valid_idx] = clf.predict(valid_x, num_iteration = clf.best_iteration_)
            test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]
        else:
            train_pred[train_idx] = clf.predict(train_x)
            train_pred_proba[train_idx] = clf.predict_proba(train_x)[:, 1]
            test_pred[valid_idx] = clf.predict(valid_x)
            test_pred_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        prediction_proba += \
                clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits

        df_feature_importance[n_fold] = pd.Series(clf.feature_importances_, index = feats)
        
        print('Fold %2d AUC : %.6f' % (n_fold, roc_auc_score(valid_y, test_pred_proba[valid_idx])))
        print('Fold %2d f1_score : %.6f' % (n_fold, f1_score(valid_y, test_pred[valid_idx])))
        folds_score.append(roc_auc_score(valid_y, test_pred_proba[valid_idx]))
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    roc_auc_train = roc_auc_score(train_df['is_promoted'], train_pred_proba)
    precision_train = precision_score(train_df['is_promoted'], train_pred, average = None)
    recall_train = recall_score(train_df['is_promoted'], train_pred, average = None)
    f1_score_train = f1_score(train_df['is_promoted'], train_pred, average = None)
    
    roc_auc_test = roc_auc_score(train_df['is_promoted'], test_pred_proba)
    precision_test = precision_score(train_df['is_promoted'], test_pred, average = None)
    recall_test = recall_score(train_df['is_promoted'], test_pred, average = None)
    f1_score_test = f1_score(train_df['is_promoted'], test_pred, average = None)
    
    print('Full F1_score %.6f' % f1_score(train_df['is_promoted'], test_pred))
    
    if logger:
        logger.info('*'*50)
        logger.info(params)
        logger.info(folds_score)
        logger.info(iteration)
        logger.info('Full F1_score %.6f' % f1_score(train_df['is_promoted'], test_pred))
        logger.info('*'*50)
    
    df_feature_importance.fillna(0, inplace = True)
    df_feature_importance['mean'] = df_feature_importance.mean(axis = 1)
    
    # Write prediction files
    if save_train_prediction:
        df_prediction = train_df[['employee_id', 'is_promoted']]
        df_prediction['Prediction_prob'] = test_pred_proba
        df_prediction['Prediction'] = test_pred
        df_prediction.to_csv(train_prediction_file_name, index = False)
        del df_prediction
        gc.collect()

    if save_test_prediction:
        df_prediction = test_df[['employee_id']]
        df_prediction['is_promoted_proba'] = prediction_proba
        df_prediction.to_csv(test_prediction_file_name, index = False)
        del df_prediction
        gc.collect()
    
    
    return df_feature_importance, \
           [roc_auc_train, roc_auc_test,
            precision_train[0], precision_test[0], precision_train[1], precision_test[1],
            recall_train[0], recall_test[0], recall_train[1], recall_test[1], f1_score_train[0], f1_score_train[1],
           f1_score_test[0], f1_score_test[1], 0]


In [27]:
logging = setup_logging()
logger = logging.getLogger('ETC')
etc_params = {'n_estimators': 1000,
             "min_samples_leaf": 5,
             'bootstrap': True}

In [ ]:
estimator = ExtraTreesClassifier()
for iteration, random_state in product([1, 2, 3], [11, 111, 786]):
    feature_importance, scor = cv_scores(estimator, df, num_folds = 5, 
                                     params = etc_params, stratified = False, random_state = random_state,
                                     train_prediction_file_name = f'../utility/X_train_etc_lb_false_{iteration}_{random_state}.csv',
                                     test_prediction_file_name = f'../utility/X_test_etc_lb_false_{iteration}_{random_state}.csv',
                                         iteration = f'iter:{iteration}, random states:{random_state}',
                                         logger = logger, lgbm_clf = False)

Starting Classifier. Train shape: (54802, 211), test shape: (23490, 211)
Fold 0 started at Sun Sep 16 15:37:56 2018
Fold  0 AUC : 0.869304
Fold  0 f1_score : 0.135354
Fold 1 started at Sun Sep 16 15:48:35 2018
Fold  1 AUC : 0.885445
Fold  1 f1_score : 0.144076
Fold 2 started at Sun Sep 16 16:00:17 2018
Fold  2 AUC : 0.864978
Fold  2 f1_score : 0.141898
Fold 3 started at Sun Sep 16 16:11:43 2018
Fold  3 AUC : 0.869138
Fold  3 f1_score : 0.146535
Fold 4 started at Sun Sep 16 16:18:49 2018
Fold  4 AUC : 0.875919
Fold  4 f1_score : 0.166344
Full F1_score 0.146921
2018-09-16 16:23:30 - ETC - INFO - **************************************************
2018-09-16 16:23:30 - ETC - INFO - {'n_estimators': 1000, 'min_samples_leaf': 5, 'bootstrap': True}
2018-09-16 16:23:30 - ETC - INFO - [0.8693040322414551, 0.8854452661893774, 0.864978037090568, 0.8691375721257917, 0.8759186188543323]
2018-09-16 16:23:30 - ETC - INFO - iter:1, random states:11
2018-09-16 16:23:30 - ETC - INFO - Full F1_score 0.14